In [2]:
import pandas as pd
import numpy as np

In [2]:
# Load Data
basic_address = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_address = 'https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_address = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

akas = pd.read_csv(akas_address, sep='\t', low_memory=False)
display(akas.head())

basics = pd.read_csv(basic_address, sep='\t', low_memory=False)
display(basics.head())

ratings = pd.read_csv(ratings_address, sep='\t', low_memory=False)
display(ratings.head())

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


,tconst,averageRating,numVotes
0,tt0000001,5.7,1910
1,tt0000002,5.8,256
2,tt0000003,6.5,1715
3,tt0000004,5.6,169
4,tt0000005,6.2,2529


In [4]:
akas = pd.read_csv('Data/title_akas.csv.gz')
basics = pd.read_csv('Data/title_basics.csv.gz')
ratings = pd.read_csv('Data/title_ratings.csv.gz')

# Start EDA

In [5]:
# Some light EDA
print('Akas')
display(akas.info())
print('Basics')
display(basics.info())
print('Ratings')
display(ratings.info())

Akas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351440 entries, 0 to 1351439
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1351440 non-null  object 
 1   ordering         1351440 non-null  int64  
 2   title            1351440 non-null  object 
 3   region           1351440 non-null  object 
 4   language         3704 non-null     object 
 5   types            964227 non-null   object 
 6   attributes       45020 non-null    object 
 7   isOriginalTitle  1350065 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 82.5+ MB


None

Basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79682 entries, 0 to 79681
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          79682 non-null  object 
 1   titleType       79682 non-null  object 
 2   primaryTitle    79682 non-null  object 
 3   originalTitle   79682 non-null  object 
 4   isAdult         79682 non-null  int64  
 5   startYear       79682 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  79682 non-null  int64  
 8   genres          79682 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.5+ MB


None

Ratings
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261370 entries, 0 to 1261369
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1261370 non-null  object 
 1   averageRating  1261370 non-null  float64
 2   numVotes       1261370 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.9+ MB


None

# Data Cleaning

In [6]:
# replace Ns with nans
basics = basics.replace('\\N', np.nan)
akas = akas.replace('\\N', np.nan)
ratings = ratings.replace('\\N', np.nan)

In [7]:
# Cleaning basics
basics = basics.dropna(subset=['genres','runtimeMinutes', 'startYear'])
basics['startYear'] = basics['startYear'].astype(int)

# Filter for recent fictional movies released in the US

In [8]:
# Filter basics for recent fiction movies
is_movie = basics['titleType'] == 'movie'
is_doc = basics['genres'].str.contains('documentary', case=False)
is_current = (basics['startYear'] >= 2000) & (basics['startYear'] <= 2021)
basics = basics.loc[is_movie
                    & ~is_doc
                    & is_current]

basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79682 entries, 0 to 79681
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          79682 non-null  object 
 1   titleType       79682 non-null  object 
 2   primaryTitle    79682 non-null  object 
 3   originalTitle   79682 non-null  object 
 4   isAdult         79682 non-null  int64  
 5   startYear       79682 non-null  int32  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  79682 non-null  int64  
 8   genres          79682 non-null  object 
dtypes: float64(1), int32(1), int64(2), object(5)
memory usage: 5.8+ MB


In [9]:
# Check akas region encoding
akas['region'].unique()

array(['US'], dtype=object)

In [10]:
# filter for US movies
akas = akas.loc[akas['region'] == 'US']

In [11]:
# Use akas to filter basics to remove non-US movies

us_movies = basics['tconst'].isin(akas['titleId'])
basics = basics[us_movies]

In [12]:
# Use basics to filter akas and ratings
akas = akas[akas['titleId'].isin(basics['tconst'])]
ratings = ratings[ratings['tconst'].isin(basics['tconst'])]

In [13]:
# Check the tables after cleaning/filtering
print('Akas')
display(akas.info())
print('Basics')
display(basics.info())
print('Ratings')
display(ratings.info())

Akas
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88979 entries, 34442 to 1351431
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   titleId          88979 non-null  object 
 1   ordering         88979 non-null  int64  
 2   title            88979 non-null  object 
 3   region           88979 non-null  object 
 4   language         892 non-null    object 
 5   types            82639 non-null  object 
 6   attributes       4159 non-null   object 
 7   isOriginalTitle  88979 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 6.1+ MB


None

Basics
<class 'pandas.core.frame.DataFrame'>
Int64Index: 79682 entries, 0 to 79681
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          79682 non-null  object 
 1   titleType       79682 non-null  object 
 2   primaryTitle    79682 non-null  object 
 3   originalTitle   79682 non-null  object 
 4   isAdult         79682 non-null  int64  
 5   startYear       79682 non-null  int32  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  79682 non-null  int64  
 8   genres          79682 non-null  object 
dtypes: float64(1), int32(1), int64(2), object(5)
memory usage: 5.8+ MB


None

Ratings
<class 'pandas.core.frame.DataFrame'>
Int64Index: 66093 entries, 17879 to 1261348
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         66093 non-null  object 
 1   averageRating  66093 non-null  float64
 2   numVotes       66093 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.0+ MB


None

# Save new tables

In [14]:
# Save the new tables
import os
os.makedirs('Data/', exist_ok=True)

basics.to_csv('Data/title_basics.csv.gz', compression='gzip', index=False)
akas.to_csv('Data/title_akas.csv.gz', compression='gzip', index=False)
ratings.to_csv('Data/title_ratings.csv.gz', compression='gzip', index=False)